# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# File to read
data = "../WeatherPy\output"

# Read Data File and store into Pandas DataFrames
weather_data = pd.read_csv(data)
weather_data


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hilo,19.7297,-155.0900,82.27,82.0,56.0,7.00,US,1.670274e+09
1,1,kologriv,58.8275,44.3178,11.61,88.0,100.0,6.06,RU,1.670274e+09
2,2,cabo san lucas,22.8909,-109.9124,85.51,55.0,20.0,4.61,MX,1.670274e+09
3,3,torbay,47.6666,-52.7314,40.26,95.0,100.0,1.14,CA,1.670274e+09
4,4,jamestown,42.0970,-79.2353,44.55,48.0,100.0,8.86,US,1.670274e+09
...,...,...,...,...,...,...,...,...,...,...
603,603,albanel,48.8832,-72.4487,33.04,76.0,99.0,11.45,CA,1.670275e+09
604,604,port blair,11.6667,92.7500,80.58,83.0,100.0,20.76,IN,1.670275e+09
605,605,hirna,9.2167,41.1000,58.12,39.0,40.0,2.17,ET,1.670275e+09
606,606,micheweni,-4.9667,39.8333,78.66,80.0,85.0,5.32,TZ,1.670274e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
len(weather_data)
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608 entries, 0 to 607
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  608 non-null    int64  
 1   City        608 non-null    object 
 2   Lat         558 non-null    float64
 3   Lng         558 non-null    float64
 4   Max Temp    558 non-null    float64
 5   Humidity    558 non-null    float64
 6   Cloudiness  558 non-null    float64
 7   Wind Speed  558 non-null    float64
 8   Country     553 non-null    object 
 9   Date        558 non-null    float64
dtypes: float64(7), int64(1), object(2)
memory usage: 47.6+ KB


In [4]:
new = weather_data.dropna()

In [5]:
len(new)


553

In [6]:
# Configure gmaps with unique API key
gmaps.configure(api_key=g_key)

In [7]:
# Use the Lat and Lng as locations and Humidity as the weight.

subset = new[['Lat', 'Lng']]
coordinates = [tuple(x) for x in subset.to_numpy()]


In [8]:
# Customize the size of the figure

fig_1 = gmaps.figure(center = [0,0] ,zoom_level = 2)

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig_1.add_layer(markers)
fig_1


Figure(layout=FigureLayout(height='420px'))

In [9]:
humidity = new["Humidity"].astype(float)

In [10]:
# Customize the size of the figure

# Plot Heatmap
fig_2 = gmaps.figure(center = [0,0] ,zoom_level = 2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity,
                               dissipating=False, max_intensity=50,
                               point_radius = 2)


# Add layer
fig_2.add_layer(heat_layer)

# Display figure
fig_2

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
# A max temperature higher than 80 degrees but lower than 100
# Wind speed less than 10 m/s
# Zero cloudiness

preferences = new.loc[(new['Cloudiness']<40) & (new['Max Temp']>=70) & (new['Max Temp']<=100) & (new['Wind Speed']<=15) ]
len(preferences)

53

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
#Create a new DataFrame called hotel_df to store the city, country, coordinates, and humidity.

hotel_df = preferences[['City', 'Country', 'Lat', 'Lng', 'Humidity']]
hotel_df

,City,Country,Lat,Lng,Humidity
2,cabo san lucas,MX,22.8909,-109.9124,55.0
12,yulara,AU,-25.2406,130.9889,17.0
75,sancti spiritus,CU,21.9297,-79.4425,60.0
86,touros,BR,-5.1989,-35.4608,73.0
89,amalapuram,IN,16.5833,82.0167,83.0
91,georgetown,MY,5.4112,100.3354,86.0
105,jalpan,MX,21.2333,-99.4833,44.0
113,mount isa,AU,-20.7333,139.5000,53.0
124,berbera,SO,10.4396,45.0143,71.0
132,chuy,UY,-33.6971,-53.4616,66.0


In [13]:

hotel = []
hotel_df

for index, row in hotel_df.iterrows():
    location = str(row["Lat"]) + "," + str(row["Lng"])
    params = {
    "location": location,
    "keyword": "hotel",
    "radius": "5000",
    "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    
    # Start try except incase a hotel cannot be located nearby
    # extract results
    
    try:
        response = requests.get(base_url, params=params).json()
        results = response['results']
        #hotel.append(response["results"][0]["name"])
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    
    except:
        hotel.append("None Nearby")

 

C:\Users\AdminLocal\AppData\Local\Temp\ipykernel_19680\3798325744.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.loc[index, 'Hotel Name'] = results[0]['name']


In [14]:
#add "Hotel Name" column to hotel
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
2,cabo san lucas,MX,22.8909,-109.9124,55.0,Solmar Resort
12,yulara,AU,-25.2406,130.9889,17.0,Sails in the Desert
75,sancti spiritus,CU,21.9297,-79.4425,60.0,Hotel Plaza
86,touros,BR,-5.1989,-35.4608,73.0,Touros Praia Hotel
89,amalapuram,IN,16.5833,82.0167,83.0,Hotel Grand Park
91,georgetown,MY,5.4112,100.3354,86.0,SAVV Hotel
105,jalpan,MX,21.2333,-99.4833,44.0,AVE Suites Campestres Jalpan
113,mount isa,AU,-20.7333,139.5000,53.0,Redearth Boutique Hotel
124,berbera,SO,10.4396,45.0143,71.0,Damal Hotel
132,chuy,UY,-33.6971,-53.4616,66.0,Hotel Internacional


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map

#markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig_2.add_layer(markers)

# Display figure
#fig_2


# Create gmaps figure variables
fig_3 = gmaps.figure(center = [0,0] ,zoom_level = 2)

# Add heatlayer
heat_layer = gmaps.heatmap_layer(coordinates, weights= new["Humidity"].astype(float),max_intensity = 100, dissipating=False, point_radius=2)
fig_3.add_layer(heat_layer)

# Add markers and info box
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig_3.add_layer(markers)

#print the figure
fig_3


Figure(layout=FigureLayout(height='420px'))